In [ ]:
import os

In [ ]:
path = 'C:\\Users\\raven\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python37\\site-packages'

In [ ]:
os.chdir(path)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import random
import numpy as np
import math
from scipy.special import softmax
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [ ]:
class act_model(nn.Module):
    def __init__(self,inp,hidden,output):
        super(act_model, self).__init__()
        self.fc1 = nn.Linear(inp, hidden, bias=True)
        self.fc2 = nn.Linear(hidden, hidden, bias=True)
        self.fc3 = nn.Linear(hidden, output, bias=True)
        self.fc8 = nn.Tanh()
        self.fc9 = nn.Sigmoid()
        self.fc10 = nn.Softmax()
        self.fc11 = nn.ReLU()
        self.fc12 = nn.LeakyReLU()
        
    def forward(self,x):
        out = self.fc12(self.fc1(x))
        out = self.fc12(self.fc2(out))
        out = self.fc3(out)
        
        return out

In [ ]:
def temp_dis_reward(reward_list,gamma):
    reward_list.reverse()
    dis_reward_list = [reward_list[0]]
    for i in range(1,len(reward_list)):
        dis_reward_list.append(reward_list[i] + gamma*dis_reward_list[i-1])
    dis_reward_list.reverse()
    return dis_reward_list

In [ ]:
inp = 4
hid = 24
out = 2

In [ ]:
model = act_model(inp,hid,out)

In [ ]:
out_list = []
loss_list = []
mse = nn.MSELoss()
bce = nn.BCELoss()
nll = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(),lr=.0001)

In [ ]:
env = gym.make('CartPole-v0')
gamma = .9
epsilon = .2
epochs = 1000
total_reward_list = []
tot_loss = []
buffer = []

for epoch in range(epochs):
    observation = env.reset()
    observation_list = []
    old_q_list = []
    reward_list = []
    max_q_list = []
    max_q_act_ind_list = []
    tot_reward = 0
    for t in range(100):
        observation_list.append(observation)
        
        q_action = model(torch.FloatTensor(observation))   #Q(s)
        
        max_q_act_ind = np.argmax(q_action.detach().numpy())  #Q(s,a)
        
        observation, reward, done, info = env.step(max_q_act_ind)
        tot_reward += reward
        
        max_q_list.append(q_action[max_q_act_ind])
        max_q_act_ind_list.append(max_q_act_ind)
        reward_list.append(reward)

        if done:
            reward_list[-1] = -10
            print("Epoch: {} timesteps: {}  Reward: {} ".format(epoch,t+1,tot_reward))
            break       
    
    dis_reward_list = temp_dis_reward(reward_list,gamma)
    dis_old_q_list = temp_dis_reward(max_q_list,gamma)
    max_q_list = temp_dis_reward(max_q_list,gamma)
    
    #print(dis_reward_list)
    ten_dis_rew_list = []
    ten_q_list = []
    for i in range(len(dis_reward_list)):
        ind = max_q_act_ind_list[i]

        one_hot_reward = torch.zeros(2)
        one_hot_reward[ind] = dis_reward_list[i]

        one_hot_q = torch.zeros(2)
        one_hot_q[ind] = max_q_list[i]

        ten_dis_rew_list.append(one_hot_reward)
        ten_q_list.append(one_hot_q)
    
    
    ten_dis_rew_list = torch.stack(ten_dis_rew_list)
    ten_q_list = torch.stack(ten_q_list)
    
    model.zero_grad()
    optimizer.zero_grad()
    loss = mse(ten_dis_rew_list,ten_q_list)
    loss.backward(retain_graph=True)
    optimizer.step() 
        
    total_reward_list.append(tot_reward)   
    tot_loss.append(loss.detach())
       
env.close()

In [ ]:
plt.plot([i for i in range(len(total_reward_list))],total_reward_list)

In [ ]:
plt.plot([i for i in range(len(tot_loss))],tot_loss)

In [ ]:
print(sum(total_reward_list)/len(total_reward_list))

In [ ]:
env = gym.make('CartPole-v0')
gamma = .9
total_reward = 0
total_reward_list = []
observation = env.reset()
for t in range(100):
    env.render()
    old_action = model(torch.FloatTensor(observation))
    old_max_act_ind = np.argmax(old_action.detach().numpy())

    observation, reward, done, info = env.step(old_max_act_ind)
    total_reward+=reward
    if done:
        print("{} timesteps. Reward: {} ".format(t+1,total_reward))
        break       
env.close()